In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [ ]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [ ]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )

In [ ]:
grouped = train.groupby("인적사고")

In [ ]:
def cosine_similarity(a, b):
    """코사인 유사도 계산"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [ ]:
res = {}
jaccard_res = []

# 자카드 유사도 함수
def jaccard_similarity(text1, text2):
    """자카드 유사도 계산"""
    set1, set2 = set(text1.split()), set(text2.split())  # 단어 집합 생성
    intersection = len(set1.intersection(set2))  # 교집합 크기
    union = len(set1.union(set2))  # 합집합 크기
    return intersection / union if union != 0 else 0

for name, group in tqdm(grouped):
    plans = group["재발방지대책 및 향후조치계획"].tolist()
    n = len(plans)
    
    # 유사도 행렬 초기화
    similarity_matrix = np.zeros((n, n))
    
    # 이중 for문을 통해 자카드 유사도 계산
    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = jaccard_similarity(plans[i], plans[j])
    
    # 각 텍스트별 평균 유사도 계산
    avg_similarity = similarity_matrix.mean(axis=1)
    
    # 평균 유사도가 가장 높은 인덱스 찾기
    best_idx = avg_similarity.argmax()
    
    # 선택된 텍스트의 유사도 결과 저장
    jaccard_res += similarity_matrix[best_idx].tolist()
    res[name] = plans[best_idx]


In [ ]:
arr = jaccard_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
res

In [ ]:
import numpy as np
from tqdm import tqdm

# 자카드 유사도 함수
def jaccard_similarity(text1, text2):
    """자카드 유사도 계산"""
    set1, set2 = set(text1.split()), set(text2.split())  # 단어 집합 생성
    intersection = len(set1.intersection(set2))  # 교집합 크기
    union = len(set1.union(set2))  # 합집합 크기
    return intersection / union if union != 0 else 0

grouped = train.groupby(["인적사고", "작업프로세스"])  # 그룹화
group_avg_similarities = {}  # 그룹별 평균 유사도를 저장할 딕셔너리

for name, group in tqdm(grouped):
    plans = group["재발방지대책 및 향후조치계획"].tolist()
    n = len(plans)
    
    if n < 2:
        # 한 개의 항목만 있는 경우 유사도는 1로 간주
        group_avg_similarities[name] = 1.0
        continue

    # 유사도 행렬 초기화
    similarity_matrix = np.zeros((n, n))
    
    # 이중 for문을 통해 자카드 유사도 계산
    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = jaccard_similarity(plans[i], plans[j])
    
    # 그룹 내 모든 유사도의 평균 계산
    avg_similarity = similarity_matrix[np.triu_indices(n, k=1)].mean()
    
    # 그룹별 평균 유사도 저장
    group_avg_similarities[name] = avg_similarity

# 유사도 값 리스트 생성
arr = list(group_avg_similarities.values())

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")


In [ ]:
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 자카드 유사도 함수
def jaccard_similarity(text1, text2):
    """자카드 유사도 계산"""
    set1, set2 = set(text1.split()), set(text2.split())  # 단어 집합 생성
    intersection = len(set1.intersection(set2))  # 교집합 크기
    union = len(set1.union(set2))  # 합집합 크기
    return intersection / union if union != 0 else 0

# 코사인 유사도 함수
def cosine_similarity(a, b):
    """코사인 유사도 계산"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

# SBERT 임베딩 모델 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

grouped = train.groupby(["인적사고"])  # 그룹화

group_avg_similarities = {}  # 그룹별 평균 유사도를 저장할 딕셔너리

def calculate_similarities(plans):
    """주어진 문장 리스트에 대해 자카드 및 코사인 유사도를 계산"""
    n = len(plans)
    if n < 2:
        return 1.0, 1.0  # 단일 항목 그룹은 유사도 1.0으로 처리
    
    # 자카드 유사도 행렬
    jaccard_matrix = np.zeros((n, n))
    
    # SBERT를 활용한 임베딩 벡터 계산
    embeddings = model.encode(plans, convert_to_numpy=True)
    
    # 코사인 유사도 행렬
    cosine_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            jaccard_matrix[i, j] = jaccard_similarity(plans[i], plans[j])
            cosine_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])
    
    avg_jaccard = jaccard_matrix[np.triu_indices(n, k=1)].mean()
    avg_cosine = cosine_matrix[np.triu_indices(n, k=1)].mean()
    
    return avg_jaccard, avg_cosine

# 각 그룹에 대해 평균 유사도 계산
for name, group in tqdm(grouped):
    plans = group["재발방지대책 및 향후조치계획"].tolist()
    avg_jaccard, avg_cosine = calculate_similarities(plans)
    group_avg_similarities[name] = {'jaccard': avg_jaccard, 'cosine': avg_cosine}

# 결과 저장 및 출력
jaccard_arr = [v['jaccard'] for v in group_avg_similarities.values()]
cosine_arr = [v['cosine'] for v in group_avg_similarities.values()]

# 0.1 단위 구간 설정
bins = np.arange(0, 1.1, 0.1)

# 히스토그램 계산
jaccard_hist, _ = np.histogram(jaccard_arr, bins=bins)
cosine_hist, _ = np.histogram(cosine_arr, bins=bins)

# 결과 출력
print("자카드 유사도 분포:")
for i in range(len(jaccard_hist)):
    print(f"Range {bins[i]:.1f} - {bins[i+1]:.1f}: {jaccard_hist[i]}개")

print("\n코사인 유사도 분포:")
for i in range(len(cosine_hist)):
    print(f"Range {bins[i]:.1f} - {bins[i+1]:.1f}: {cosine_hist[i]}개")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 그룹별 자카드 및 코사인 유사도 데이터를 저장
group_names = list(group_avg_similarities.keys())
jaccard_values = [v['jaccard'] for v in group_avg_similarities.values()]
cosine_values = [v['cosine'] for v in group_avg_similarities.values()]

# 시각화를 위한 설정
plt.figure(figsize=(12, 6))

# 자카드 유사도 분포
plt.subplot(1, 2, 1)
plt.barh(group_names, jaccard_values, color='blue', alpha=0.6)
plt.xlabel("자카드 유사도")
plt.ylabel("그룹")
plt.title("그룹별 자카드 유사도")

# 코사인 유사도 분포
plt.subplot(1, 2, 2)
plt.barh(group_names, cosine_values, color='green', alpha=0.6)
plt.xlabel("코사인 유사도")
plt.title("그룹별 코사인 유사도")

plt.tight_layout()
plt.show()
